# The Model
$
\huge y_{i} 
= \alpha + \beta_1{x_{i1}} + \beta_2{x_{i2}} +...+ \beta_k{x_{ik}} + \epsilon_{i}
$

* 예) minutes = $\alpha$ + $\beta_1$friends + $\beta_2$work_hours + $\beta_3$phd + $\epsilon$
* Simple Linear Regression의 확장
* 입력 변수가 여러개 ($x_{i1},x_{i2},...,x_{ik}$)
* 회귀계수도 여러개 ($\alpha, \beta_1, \beta_2, ...,\beta_k$)
  + 예측에 필요한 계수. 머신러닝을 통해 구해야 할 값


____
##### Simple Linear Regression

  + $y_{i} = \alpha + \beta {x_{i}} + \epsilon_{i} $
  + 회귀계수: $\alpha$, $\beta$
  + 하나의 입력값 ($x_i$)
____


## Vector 표현
* 입력 변수와 회귀계수가 여러개 &rarr; Vector로 다루면 편리하다
  + beta = [alpha, beta_1, beta_2, ..., beta_k]
  + x_i = [1, x_i1, x_i2, ..., x_ik]


## 예측값
* Vector의 dot 연산 이용
  - x_i 벡터의 첫번째 요소로 상수값 1을 사용한 이유
* 회귀계수 벡터를 구한 상태에서, 입력변수 벡터가 주어졌을때의 예측값
  - dot(x_i, beta): $\alpha + \beta_1{x_{i1}} + \beta_2{x_{i2}} +...+ \beta_k{x_{ik}}$

In [15]:
# 'scratch' package가 설치된 디렉토리로 작업 디렉토리 바꿈 (package를 import하기 위함)
import os, sys
os.chdir('..')

from scratch.linear_algebra import dot, Vector

def predict(x: Vector, beta: Vector) -> float:
    """assumes that the first element of x is 1"""
    return dot(x, beta)

In [16]:
# Example - x_i
x_i= [1,    # constant term
      49,   # number of friends
      4,    # work hours per day
      0]    # doesn't have PhD

## Further Assumptions of the Least Squares Model
* 입력변수($x_{ij}$)는 다른 입력변수들의 Weighted Sum(선형결합)으로 구할 수 없다 (Linearly Independent)
* 입력변수($x_{ij}$)는 오류($\epsilon_{i}$)와 서로 상관관계가 없다
* 위의 두 가정이 만족되지 않으면 정확한 결과를 얻지 못한다
  + 실제로 위의 두 가정을 제대로 만족하는 모형을 만들기는 쉽지 않다
  + "정확한" 결과를 얻지는 못하지만 "괜찮은" 수준의 결과는 얻을 수 있다

# Fitting the Model
* Simple Linear Regression의 확장
* Gradient Descent 알고리즘을 이용하여 회귀계수를 구할 것임
* 다만, 입력변수와 회귀계수가 모두 벡터로 표현되기 때문에 일부 함수의 구현이 달라진다

In [17]:
from typing import List

def error(x: Vector, y: float, beta: Vector) -> float:
    return predict(x, beta) - y

def squared_error(x: Vector, y: float, beta: Vector) -> float:
    return error(x, y, beta) ** 2

x = [1, 2, 3]
y = 30
beta = [4, 4, 4]  # so prediction = 4 + 8 + 12 = 24

assert error(x, y, beta) == -6
assert squared_error(x, y, beta) == 36

### Gradient 함수
* [Remind] Simple Linear Regression에서
  + $SSE = \sum_{i=1}^{n}(y_i - \alpha - \beta x_i)^2$
  + $\frac{\partial SSE}{\partial \alpha} = 2(y_i- \alpha - \beta x_i) = 2\epsilon_i$
  + $\frac{\partial SSE}{\partial \beta} = 2(y_i- \alpha - \beta x_i)x_i = 2\epsilon_i x_i$
* Multiple Regression에서는 $\alpha$ 값이 벡터 beta에 포함되도록 모형을 만들었음
  + beta = [alpha, beta_1, beta_2, ..., beta_k]
    - 입력 벡터 $x_i$의 첫번째 값이 1이므로
    - $\frac{\partial SSE}{\partial \alpha} = 2(y_i- \alpha - \beta x_i) = 2\epsilon_i$
    - $\frac{\partial SSE}{\partial \beta} = 2(y_i- \alpha - \beta x_i)x_i = 2\epsilon_i x_i$
    - $x_i=1$ 일 때, $2\epsilon_i x_i = 2\epsilon_i=\frac{\partial SSE}{\partial \alpha}$ 
  + 따라서, $\beta$에 대한 Gradient만 있으면 됨

In [18]:
def sqerror_gradient(x: Vector, y: float, beta: Vector) -> Vector:
    err = error(x, y, beta)
    return [2 * err * x_i for x_i in x]

assert sqerror_gradient(x, y, beta) == [-12, -24, -36]

### Fitting (Least Squares Fit)
* Gradient Descent 알고리즘을 이용하여 오류 제곱의 합이 최소가 되는 $\beta$(벡터)를 찾는다

In [19]:
import random
import tqdm
from scratch.linear_algebra import vector_mean
from scratch.gradient_descent import gradient_step


def least_squares_fit(xs: List[Vector],
                      ys: List[float],
                      learning_rate: float = 0.001,
                      num_steps: int = 1000,
                      batch_size: int = 1) -> Vector:
    """
    Find the beta that minimizes the sum of squared errors
    assuming the model y = dot(x, beta).
    """
    # Start with a random guess
    guess = [random.random() for _ in xs[0]]

    for _ in tqdm.trange(num_steps, desc="least squares fit"):
        for start in range(0, len(xs), batch_size):
            batch_xs = xs[start:start+batch_size]
            batch_ys = ys[start:start+batch_size]

            gradient = vector_mean([sqerror_gradient(x, y, guess)
                                    for x, y in zip(batch_xs, batch_ys)])
            guess = gradient_step(guess, gradient, -learning_rate)

    return guess

In [20]:
# 1(고정값), num_friends, work_hours, phd(Yes/No)
inputs: List[List[float]] = [[1.,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],[1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],[1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],[1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],[1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],[1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]
from scratch.statistics import daily_minutes_good
from scratch.gradient_descent import gradient_step

In [21]:
random.seed(0)
# I used trial and error to choose niters and step_size.
# This will run for a while.
learning_rate = 0.001

beta = least_squares_fit(inputs, daily_minutes_good, learning_rate, 5000, 25)
assert 30.50 < beta[0] < 30.70  # constant
assert  0.96 < beta[1] <  1.00  # num friends
assert -1.89 < beta[2] < -1.85  # work hours per day
assert  0.91 < beta[3] <  0.94  # has PhD

print('Beta', beta)

least squares fit: 100%|██████████████████████████████████████████████████████████| 5000/5000 [00:06<00:00, 792.57it/s]


Beta [30.514795945185586, 0.9748274277323267, -1.8506912934343662, 0.91407780744768]


### 학습결과
* Beta: [30.514795945185586, 0.9748274277323267, -1.8506912934343662, 0.91407780744768]
* 최종 예측 모델
  + minutes = 30.51 + 0.975*friends -1.85*work_hours + 0.91*phd

In [22]:
print("100 friends, 8 work_hours, has PhD ==> Minutes online", predict([1, 100, 8, 1], beta) )

100 friends, 8 work_hours, has PhD ==> Minutes online 114.10608617839101


# Interpreting the Model
* all-else-being-equal
  + 예) 다른 모든 입력 값이 똑같을 때, 친구 1명이 추가되면 인터넷에서 보내는 시간이 1분 정도 증가한다
  + 그러나, 실제로는 입력 변수들 간에 "간섭"이 있을 수 있다
    - 예) 친구가 많고 적음에 따라 work_hours가 달라질 수 있음
  + 보정 예) num_friends x work_hours 입력 변수를 추가 (서로 간섭, 또는 영향을 미칠 수 있는 변수들의 곱)
* 친구 수가 많으면 많을수록 인터넷에서 보내는 시간이 길어질까?
  + 어느 정도까지는 길어질 수 있지만, 일정 수준(up to a point)을 넘으면 의미가 없어짐
  + 보정 예) 이러한 변수는 제곱(Square)을 취해준다
  + 제곱을 취하면 입력변수의 값이 제곱만큼 커질 수 있음 &rarr; 입력값이 커지면 회귀계수는 작아지는 방향으로 보정될 것임
    - Gradient Descent 알고리즘에 의해 오류를 0에 가깝게 만들는 방향으로 회귀계수를 조정하므로

# Goodness of Fit
* [주의] 입력 변수 추가 &rarr; R-squared 값도 커진다

In [23]:
from scratch.simple_linear_regression import total_sum_of_squares

def multiple_r_squared(xs: List[Vector], ys: Vector, beta: Vector) -> float:
    sum_of_squared_errors = sum(error(x, y, beta) ** 2
                                for x, y in zip(xs, ys))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(ys)

assert 0.67 < multiple_r_squared(inputs, daily_minutes_good, beta) < 0.68

# Digression: The Bootstrap
* 원본 데이터셋에서 $n$개의 샘플 추출 (복원추출. Sampling with replacement)

### 예 - 분포가 편향된 데이터의 통계
* 데이터셋 1: 대부분의 데이터가 100 근처에 몰려있다
* 데이터셋 2: 100 근처의 값이 하나. 나머지의 반은 0 근처에 몰려있고 반은 200 근처에 몰려있다. 
* 위의 두 데이터셋은 그 분포가 완전히 다르다
  + 하지만 median은 둘 다 100
* Bootstrap 적용 (샘플을 취해서 median을 구하기)
  + 데이터셋 1: 큰 차이가 없다
  + 데이터셋 2: 샘플링에 따라 0에 가까운 median, 또는 200에 가까운 median이 나올것이다
* 학습 데이터셋이 작거나 불완전할 때 유용하게 사용할 수 있음
  + 원본 학습 데이터셋에서 Bootstrap Sampling &rarr; 여러개의 학습 데이터셋

In [24]:
from typing import TypeVar, Callable

X = TypeVar('X')        # Generic type for data
Stat = TypeVar('Stat')  # Generic type for "statistic"

def bootstrap_sample(data: List[X]) -> List[X]:
    """randomly samples len(data) elements with replacement"""
    return [random.choice(data) for _ in data]

def bootstrap_statistic(data: List[X],
                        stats_fn: Callable[[List[X]], Stat],
                        num_samples: int) -> List[Stat]:
    """evaluates stats_fn on num_samples bootstrap samples from data"""
    return [stats_fn(bootstrap_sample(data)) for _ in range(num_samples)]

In [25]:
# 101 points all very close to 100
close_to_100 = [99.5 + random.random() for _ in range(101)]

# 101 points, 50 of them near 0, 50 of them near 200
far_from_100 = ([99.5 + random.random()] +
                [random.random() for _ in range(50)] +
                [200 + random.random() for _ in range(50)])

from scratch.statistics import median, standard_deviation

medians_close = bootstrap_statistic(close_to_100, median, 100)

medians_far = bootstrap_statistic(far_from_100, median, 100)

assert standard_deviation(medians_close) < 1
assert standard_deviation(medians_far) > 90

# Standard Errors of Regression Coefficients
* Boostrap Sampling을 이용한 회귀계수(Coefficients, $\beta_i$) 검증
* 원본 학습 데이터셋에서 Bootstrap Sampling → 여러개의 학습 데이터셋
  + 각 학습 데이터셋으로 학습(Model Fitting)한 회귀계수(Coefficients, $\beta_i$)들을 비교
  + 큰 차이가 없다면 신뢰할만한 회귀계수를 얻었다고 판단할 수 있다

In [31]:
from typing import Tuple

import datetime

def estimate_sample_beta(pairs: List[Tuple[Vector, float]]):
    x_sample = [x for x, _ in pairs]
    y_sample = [y for _, y in pairs]
    beta = least_squares_fit(x_sample, y_sample, learning_rate, 5000, 25)
    print("bootstrap sample", beta)
    return beta

random.seed(0) # so that you get the same results as me

# This will take a couple of minutes!
bootstrap_betas = bootstrap_statistic(list(zip(inputs, daily_minutes_good)),
                                      estimate_sample_beta,
                                      10) # 10번만 하자 (교재에는 100번)

bootstrap_standard_errors = [
    standard_deviation([beta[i] for beta in bootstrap_betas])
    for i in range(4)]

print('\n', 'bootstrap_standard_errors:', bootstrap_standard_errors)

least squares fit: 100%|██████████████████████████████████████████████████████████| 5000/5000 [00:05<00:00, 884.86it/s]


bootstrap sample [30.49402029547432, 1.0393791030498776, -1.9516851948558502, 0.7483721251697333]


least squares fit: 100%|██████████████████████████████████████████████████████████| 5000/5000 [00:06<00:00, 737.37it/s]


bootstrap sample [30.149963287526045, 1.0005300432763113, -2.0650380122822543, 3.177179854834797]


least squares fit: 100%|██████████████████████████████████████████████████████████| 5000/5000 [00:07<00:00, 670.83it/s]


...

bootstrap sample [29.254530450577782, 0.9763387220017684, -1.7430339427043595, 1.9944240584590935]


least squares fit: 100%|██████████████████████████████████████████████████████████| 5000/5000 [00:08<00:00, 623.27it/s]


bootstrap sample [31.649174199331632, 0.9389340937491032, -1.9733848473304205, -0.15249287969349437]


least squares fit: 100%|██████████████████████████████████████████████████████████| 5000/5000 [00:07<00:00, 640.00it/s]


bootstrap sample [30.040109260720964, 1.0531247386421572, -1.7694878560354388, 1.302971911084249]

 bootstrap_standard_errors: [1.0490224525548588, 0.08527846751503912, 0.16381146526868864, 1.2028866078811085]


## 가설 검증
* 귀무가설(Null Hupotheses): $\beta_j$는 0일까?
  + $\beta_j=0$ &rarr; ${\beta_j}{x_{ij}}=0$
  + 입력값이 무엇이든 출력에 영향을 미치지 않음 (해당 회귀계수는 의미가 없음)
* 확률모델
  + $t_j=\hat{\beta_j}/\hat{\sigma_j}$
  + $t_j$는 Degree of Freedom(자유도) $n-k$인 Student's t-distribution
    - [스튜던트-t 분포](https://ko.wikipedia.org/wiki/%EC%8A%A4%ED%8A%9C%EB%8D%98%ED%8A%B8_t_%EB%B6%84%ED%8F%AC)
    - $n$: 데이터셋 크기 (데이터 레코드 수)
    - $k$: 벡터 크기 (회귀계수의 갯수)
  + 데이터셋이 크다 &rarr; 자유도도 크다 &rarr; 자유도가 크면 스튜던트-t 분포는 표준정규분포에 근접한다
  + 우리가 머신러닝에서 다루는 데이터셋은 크기가 크기 때문에 표준정규분포를 사용해도 됨
    - $\hat{\beta_j}/\hat{\sigma_j}$ 값의 확률분포는 표준정규분포를 따른다고 가정
  + 표준정규분포
    - $\mu$(평균)=0, $\sigma$(표준편차)=1, Bell-curve
    - [표준정규분포](https://ko.wikipedia.org/wiki/%EC%A0%95%EA%B7%9C_%EB%B6%84%ED%8F%AC#%ED%91%9C%EC%A4%80_%EC%A0%95%EA%B7%9C_%EB%B6%84%ED%8F%AC)
* p-value (유의확률)
  + 귀무가설($\beta_j=0$)이 맞다고 가정할 때 얻은 결과보다 극단적인 결과가 실제로 관측될 확률
    - [유의확률](https://ko.wikipedia.org/wiki/%EC%9C%A0%EC%9D%98_%ED%99%95%EB%A5%A0)

In [30]:
print('\n', bootstrap_standard_errors)


 [1.0490224525548588, 0.08527846751503912, 0.16381146526868864, 1.2028866078811085]


In [41]:
#from scratch.probability import normal_cdf

# 저자 소스의 probability.py 파일을 import할 때 다른 코드로 인해 오류가 나타남
# probability.py의 normal_cdf() 함수를 다시 정의하여 사용하자 (Copy & Paste normal_cdf() from probability.py)
import math
def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def p_value(beta_hat_j: float, sigma_hat_j: float) -> float:
    if beta_hat_j > 0:
        # if the coefficient is positive, we need to compute twice the
        # probability of seeing an even *larger* value
        return 2 * (1 - normal_cdf(beta_hat_j / sigma_hat_j))
    else:
        # otherwise twice the probability of seeing a *smaller* value
        return 2 * normal_cdf(beta_hat_j / sigma_hat_j)

assert p_value(30.58, 1.27)   < 0.001  # constant term
assert p_value(0.972, 0.103)  < 0.001  # num_friends
assert p_value(-1.865, 0.155) < 0.001  # work_hours
assert p_value(0.923, 1.249)  > 0.4    # phd

### 해석
* phd 항목의 p-value가 크게 나옴 &rarr; not "significantly" different from 0 (별 의미없는 회귀계수)

In [42]:
print('alpha(constant term): %.4f' % p_value(30.58, 1.27))
print('beta_1(num_friends): %.4f' % p_value(0.972, 0.103))
print('beta_2(work_hours): %.4f'% p_value(-1.865, 0.155))
print('beta_3(phd): %.4f'% p_value(0.923, 1.249))

alpha(constant term): 0.0000
beta_1(num_friends): 0.0000
beta_2(work_hours): 0.0000
beta_3(phd): 0.4599


# Regularization
* 실제로 데이터를 분석할 때는 변수가 많은 데이터에 회귀 분석 모델을 적용해야 하는 경우가 발생한다.
  + 변수가 많다 &rarr; 회귀계수가 많다
  + 회귀계수가 많다 &rarr; 모델이 학습 데이터에 오버피팅할 가능성이 크다
  + 0이 아닌 회귀계수가 많을수록 모델을 해석하기 어렵다 (많다 &rarr; 복잡하다)
  + 어떠한 현상을 설명하는 것이 목표라면 수백 개의 변수로 모델을 만드는 것보다는 세 개 정도의 변수로 작은 모델을 만드는 것이 낫다
* Regularization은 $\beta_i$가 커질수록 오류(error, $\epsilon_i)$ 값에 페널티를 주는 방법
  + 오류와 패널티를 동시에 최소화하도록 유도

### Ridge Regression
* $\beta_i$의 제곱값에 비례하는 페널티를 오류에 더한다
* 예) Ridge Error: $\alpha{\beta_i}^2$
* 기본 Multiple Regression에서 오류 함수만 바뀜
  + Gradient Descent 알고리즘에 Ridge 오류 함수와 이의 Gradient 함수를 주입


In [43]:
# alpha is a *hyperparameter* controlling how harsh the penalty is
# sometimes it's called "lambda" but that already means something in Python
def ridge_penalty(beta: Vector, alpha: float) -> float:
    return alpha * dot(beta[1:], beta[1:])

def squared_error_ridge(x: Vector,
                        y: float,
                        beta: Vector,
                        alpha: float) -> float:
    """estimate error plus ridge penalty on beta"""
    return error(x, y, beta) ** 2 + ridge_penalty(beta, alpha)

In [48]:
from scratch.linear_algebra import add

def ridge_penalty_gradient(beta: Vector, alpha: float) -> Vector:
    """gradient of just the ridge penalty"""
    return [0.] + [2 * alpha * beta_j for beta_j in beta[1:]]

def sqerror_ridge_gradient(x: Vector,
                           y: float,
                           beta: Vector,
                           alpha: float) -> Vector:
    """
    the gradient corresponding to the ith squared error term
    including the ridge penalty
    """
    return add(sqerror_gradient(x, y, beta),
               ridge_penalty_gradient(beta, alpha))

def least_squares_fit_ridge(xs: List[Vector],
                            ys: List[float],
                            alpha: float,
                            learning_rate: float,
                            num_steps: int,
                            batch_size: int = 1) -> Vector:
    # Start guess with mean
    guess = [random.random() for _ in xs[0]]

    for i in range(num_steps):
        for start in range(0, len(xs), batch_size):
            batch_xs = xs[start:start+batch_size]
            batch_ys = ys[start:start+batch_size]

            gradient = vector_mean([sqerror_ridge_gradient(x, y, guess, alpha)
                                    for x, y in zip(batch_xs, batch_ys)])
            guess = gradient_step(guess, gradient, -learning_rate)

    return guess

In [49]:
random.seed(0)
beta_0 = least_squares_fit_ridge(inputs, daily_minutes_good, 0.0,  # alpha
                                 learning_rate, 5000, 25)
# [30.51, 0.97, -1.85, 0.91]
assert 5 < dot(beta_0[1:], beta_0[1:]) < 6
assert 0.67 < multiple_r_squared(inputs, daily_minutes_good, beta_0) < 0.69

beta_0_1 = least_squares_fit_ridge(inputs, daily_minutes_good, 0.1,  # alpha
                                   learning_rate, 5000, 25)
# [30.8, 0.95, -1.83, 0.54]
assert 4 < dot(beta_0_1[1:], beta_0_1[1:]) < 5
assert 0.67 < multiple_r_squared(inputs, daily_minutes_good, beta_0_1) < 0.69


beta_1 = least_squares_fit_ridge(inputs, daily_minutes_good, 1,  # alpha
                                 learning_rate, 5000, 25)
# [30.6, 0.90, -1.68, 0.10]
assert 3 < dot(beta_1[1:], beta_1[1:]) < 4
assert 0.67 < multiple_r_squared(inputs, daily_minutes_good, beta_1) < 0.69

beta_10 = least_squares_fit_ridge(inputs, daily_minutes_good,10,  # alpha
                                  learning_rate, 5000, 25)
# [28.3, 0.67, -0.90, -0.01]
assert 1 < dot(beta_10[1:], beta_10[1:]) < 2
assert 0.5 < multiple_r_squared(inputs, daily_minutes_good, beta_10) < 0.6

### Lasso Regression
* 모든 회귀계수를 최대한 0으로 만드는 방식의 페널티 적용
  - 중요한 몇 개의 회귀계수외에는 모두 0으로 수렴시킨다 (Feature Selection 효과)
  - Sparse 모델 처리에 적합 (결손이 많은 데이터셋, 값이 빈 컬럼이 많은 데이터셋)

In [47]:
def lasso_penalty(beta, alpha):
    return alpha * sum(abs(beta_i) for beta_i in beta[1:])


[참고] Ridge, Lasso, Multi-task Lasso, …
* http://scikit-learn.org/stable/modules/linear_model.html